# Different extraction distances

- COMSOL files in /Users/hkromer/02_PhD/02_Data/01_COMSOL/01_IonOptics/03.new_chamber/07.final_design/07.final_design.suppression_electrode/extraction_distance/

- HV is -100 kV, extraction distance offset is 10 mm

- bias voltage is -1.4 kV

- 1 mA ion beam current

- extraction offsets are 8, 9, 10, 11, 12, 13, 14, 15 mm

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import re
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import curve_fit

from scipy.stats import kde
from scipy import optimize

from matplotlib.ticker import NullFormatter
from matplotlib import pyplot, transforms
import matplotlib
from matplotlib.ticker import AutoMinorLocator
%matplotlib inline
import os
os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2018/bin/x86_64-darwin'
plt.rc('text', usetex=True)
plt.rc('font', weight='bold')
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.rm'] = 'Arial'
matplotlib.rcParams['mathtext.it'] = 'Arial:italic'
matplotlib.rcParams['mathtext.bf'] = 'Arial:bold'
matplotlib.rcParams['mathtext.tt'] = 'Arial'
matplotlib.rcParams['mathtext.cal'] = 'Arial'
matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']


folder = '/Users/hkromer/02_PhD/02_Data/01_COMSOL/01_IonOptics/03.new_chamber/07.final_design/07.final_design.suppression_electrode/extraction_distance/particleData/'
outfolder = '/Users/hkromer/polybox/Thesis/Chapters/FurtherNeutronGeneratorUpgrades/Figures/COMSOL_final_with_extraction_extraction_distances/'
files = os.listdir(folder)
main_figname = 'FWHM_vs_x'

# what are the sweep values
sweep_vals = [['8', '9', '10', '11'], # file 01
              ['12', '13', '14', '15']
             ]

fignames = [f.replace('.txt', '') for f in files]
datafiles = [f"{folder}{f}" for f in files]

files

In [ ]:
for ii in range(0, len(files)):
    fname = datafiles[ii]
    sweep = sweep_vals[ii]
    dic_dfs = preprocess_COMSOL_sweep(fname, sweep)
    for key in dic_dfs:
        data = dic_dfs[key]
        figname = f"{files[ii]}_{key}"
        make_plot_KDE(data, outfolder, figname, axis_lim=3)

## Function to process the COMSOL output (column headers with commas are a problem)

In [ ]:
def preprocess_COMSOL_sweep(fname, sweep_vals):
    """
    fname: csv file of COMSOL output to read
    sweep_vals: what are the sweep values in a list
    
    Returns a dictionary with the dataframes qx qy qz for each file
    """
    
    # read the data and format the , correctly
    data = pd.read_csv(fname, skiprows=8, index_col=0, nrows=10, delimiter=r'\s+', header=None)
    with open(fname, 'r') as file:
        for line in file:
            if 'Index' in line:
                cols = line.rstrip().split(',')
                break
        file.close()
    col_str = ''.join(cols)
    col_str = col_str.replace('q', ' ,q')
    col_list = col_str.split(',')
    col_list = col_list[1:]



    # read the data again, use the columns found before as the headers
    data = pd.read_csv(fname, skiprows=8, index_col=0, header=None, delimiter=r'\s+')
    data.columns = col_list

    # filter the columns
    d_sweep = {}
    # dictionary of the dataframes, each dataframe is a separate sweep value
    dic_dfs = {}
    for val in sweep_vals:
        cols = [c for c in data.columns if val in c]
        d_sweep[val] = cols

    # add each dataframe to a separate place in the dictionary
    for key in d_sweep.keys():
        this_data = data.loc[:,d_sweep[key]]
        # rename the columns in qx, qy, qz
        new_cols = []
        for c in cols:
            if 'qx' in c:
                new_cols.append('qx')
            if 'qy' in c:
                new_cols.append('qy')
            if 'qz' in c:
                new_cols.append('qz')
        this_data.columns = new_cols

        dic_dfs[key] = this_data

    return dic_dfs

## Function to make the KDE plot

In [ ]:
def make_plot_KDE(data, outfolder, figname, axis_lim):
    """
    Makes the gaussian kde and the y z plot.
    data is the dataframe with qx, qy, qz as headers and reach row is one sample data with the respecitve positions
    """
    nbins = 200
    lim = axis_lim
    #     print(data[pd.isnull(data).any(axis=1)])
    # save as csv
    data.to_csv(f'{outfolder}/{figname}.csv')
    x = data['qy'].values
    y = data['qz'].values
    my_data = np.vstack([x, y])
    k = kde.gaussian_kde(my_data)

    xi, yi = np.mgrid[-lim:lim:nbins*1j, -lim:lim:nbins*1j]
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))
#     print(zi.reshape(xi.shape))
    # plot
    fs = 20

    f = plt.figure(1, figsize=(9, 9))

    nullfmt = NullFormatter()         # no labels
    # definitions for the axes
    left, width = 0.12, 0.65
    bottom, height = 0.12, 0.65
    rect_scatter = [left, bottom, width, height]
    axScatter = plt.axes(rect_scatter)
    p = axScatter.pcolormesh(xi, yi, zi.reshape(xi.shape), shading='gouraud', cmap=plt.cm.jet)
    plt.axis('equal')
    plt.xlabel(r'\textbf{y [mm]}', fontsize=fs)
    plt.ylabel(r'\textbf{z [mm]}', fontsize=fs)
    axScatter.tick_params('x', colors='black', labelsize=fs-2)
    axScatter.tick_params('y', colors='black', labelsize=fs-2)
    plt.yticks(np.arange(-lim,lim+1,1))
    plt.xticks(np.arange(-lim,lim+1,1))


    left = 0.8
    bottom = 0.12
    width = 0.05
    height = 0.65
    cax = f.add_axes([left, bottom, width, height])
    cbar = f.colorbar(p, cax, ticks=[0,0.05,0.1,0.15,0.20,0.25,0.30,0.35,0.40])
#     print(np.max(zi))
    cbar.ax.tick_params(labelsize=fs-2)
    plt.savefig(f'{outfolder}/{figname}.pdf')

    plt.show()

    # query from the estimated pdf
    qry_eval = np.linspace(-lim,lim,100)
    eval_x = [k.evaluate([x,0])[0] for x in qry_eval] # for z = 0 in the upper plot
    eval_y = [k.evaluate([0,y])[0] for y in qry_eval] # for y = 0 in the upper plot

    # fit FWHM
    def gauss(x, a, x0, sigma):
        return a * np.exp(-(x-x0)**2/(2*sigma**2))


    p0 = [1, 1, 1] # Initial guess for the parameters
    # fit for parallel to x axis
    X = qry_eval
    Y = eval_x
    popt, pcov = curve_fit(gauss, X, Y, p0, maxfev=30000)
    A_x, x0_x, sigma_x = popt
    Y_fit_x = gauss(X, *popt)
    FWHM_x = 2 * sigma_x * np.sqrt(2 * np.log(2))  # FWHM
    print(f'FWHM in x: {FWHM_x}')

    # fit for parallel to y axis
    X = qry_eval
    Y = eval_y
    popt, pcov = curve_fit(gauss, X, Y, p0, maxfev=30000)
    A_y, x0_y, sigma_y = popt
    FWHM_y = 2 * sigma_y * np.sqrt(2 * np.log(2))  # FWHM
    print(f'FWHM in y: {FWHM_y}')
    Y_fit_y = gauss(X, *popt)
#     print(gauss(x0_y, *popt), A_y)

    f = plt.figure(1, figsize=(8, 6.4))
    ylims = (-0.01, 0.30)
    # top plot
    plt.subplot(2, 1, 1)
    plt.plot(qry_eval, eval_x, color='darkblue', linewidth=2.0, label='z=0')
    plt.plot(qry_eval, Y_fit_x, color='darkred',linewidth=2.0, label='Gaussian fit')
    plt.plot([-FWHM_x/2,FWHM_x/2],[A_x/2,A_x/2], '--', color='black')
    plt.text(-0.7, .06, f'FWHM = {FWHM_x:.2f} mm', fontsize=14)
    plt.xlabel(r'\textbf{y [mm]}', fontsize=fs)
    plt.ylabel(r'\textbf{Estimated PDF [-]}', fontsize=fs)
    ax = plt.gca()
    ax.tick_params('x', colors='black', labelsize=fs-2)
    ax.tick_params('y', colors='black', labelsize=fs-2)
    minor_locator = AutoMinorLocator(2)
    ax.xaxis.set_minor_locator(minor_locator)
    minor_locator = AutoMinorLocator(2)
    ax.yaxis.set_minor_locator(minor_locator)
    ax.grid(b=True, which='major', linestyle='-')
    ax.grid(b=True, which='minor', linestyle='--')
    plt.yticks(np.arange(0,0.4,0.1))
    plt.xticks(np.arange(-3,4,1))
    leg1 = plt.legend(loc="best",  fontsize=12)
    plt.ylim(ylims)
    # bottom plot
    plt.subplot(2, 1, 2)
    plt.plot(qry_eval, eval_y, color='darkorange', linewidth=2.0, label='y=0')
    plt.plot(qry_eval, Y_fit_y, color='darkred',linewidth=2.0, label='Gaussian fit')
    plt.plot([-FWHM_y/2+x0_y,FWHM_y/2+x0_y],[gauss(-FWHM_y/2+x0_y,*popt),gauss(FWHM_y/2+x0_y,*popt)], '--', color='black')
  
    plt.text(-0.3, .06, f'FWHM = {FWHM_y:.2f} mm', fontsize=14)
    plt.xlabel(r'\textbf{z [mm]}', fontsize=fs)
    plt.ylabel(r'\textbf{Estimated PDF [-]}', fontsize=fs)
    ax = plt.gca()
    ax.tick_params('x', colors='black', labelsize=fs-2)
    ax.tick_params('y', colors='black', labelsize=fs-2)
    minor_locator = AutoMinorLocator(2)
    ax.xaxis.set_minor_locator(minor_locator)
    minor_locator = AutoMinorLocator(2)
    ax.yaxis.set_minor_locator(minor_locator)
    ax.grid(b=True, which='major', linestyle='-')
    ax.grid(b=True, which='minor', linestyle='--')
    plt.yticks(np.arange(0,0.4,0.1))
    plt.xticks(np.arange(-3,4,1))
    leg1 = plt.legend(loc="best",  fontsize=12)
    # leg1.set_title(r"Experiment number", prop = {'size': 12})
    plt.tight_layout()
    plt.ylim(ylims)
    plt.savefig(f'{outfolder}/{figname}_YZ.pdf')
    plt.show()

In [ ]:
def get_FWHM(x, k, r, ax):
    """
    Get the FWHMs parallel to the axis: ax
    """
    assert ax in ['x', 'y']
    if ax == 'x':
        y = [k.evaluate([y,r])[0] for y in x]
    else:
        y = [k.evaluate([r,y])[0] for y in x]
    
    if np.max(y) > 1e-4:
        def find_nearest(array, value):

            array = np.asarray(array)
            dis = (np.abs(array - value))
#             if len(dis) < 5:
#                 print(y, array, value)
            idx = dis.argmin()

            return idx, array[idx]

        Y_max = np.max(y)
        Y_max_half = Y_max/2.0
    #     print(Y_max, Y_max_half)
        # x point on positive and side
        idx_0 = find_nearest(x, Y_max)[0]
        idx_pos_max_half = find_nearest(y[idx_0:], Y_max_half)[0]
        idx_neg_max_half = find_nearest(y[:idx_0], Y_max_half)[0]
    #     print(find_nearest(eval_x[idx_0:], Y_max_half))
        # FWHM
        FWHM = x[idx_pos_max_half+idx_0]-x[idx_neg_max_half]

    #     return x[idx_neg_max_half], x[idx_pos_max_half+idx_0], FWHM
        return FWHM
    else:
        return 0

def get_max_FWHMs(data, nbins=400, eval_points=300, lim=4):
    """
    returns the maximum values for the FWHM paralell to x and y direction. X is corresponding to parallel to y, z is parallel to y
    """
    x = data['qy'].values
    y = data['qz'].values
    my_data = np.vstack([x, y])
    k = kde.gaussian_kde(my_data)

    xi, yi = np.mgrid[-lim:lim:nbins*1j, -lim:lim:nbins*1j]
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))

    eval_points 
    qry_eval = np.linspace(-lim,lim,eval_points)

    df = pd.DataFrame({'qry_eval': qry_eval})
    df['FWHM_z'] = df['qry_eval'].apply(lambda r: get_FWHM(qry_eval, k, r, ax='y'))
    df['FWHM_y'] = df['qry_eval'].apply(lambda r: get_FWHM(qry_eval, k, r, ax='x'))
    FWHM_y = np.max(df['FWHM_y'] )
    FWHM_z = np.max(df['FWHM_z'] )
    
    return pd.Series([FWHM_y, FWHM_z], index=['FWHM_y', 'FWHM_z'])

In [ ]:
dic_dfs = {}
for ii in range(0, len(files)):
    fname = datafiles[ii]
    sweep = sweep_vals[ii]
    dic = preprocess_COMSOL_sweep(fname, sweep)
    dic_dfs.update(dic)

dic_dfs.keys()

In [ ]:
df = pd.DataFrame()

for key in dic_dfs:
    print(fname, key)
    
    data = dic_dfs[key]
    s = get_max_FWHMs(data, nbins=400, eval_points=300, lim=4)
    df[f"{key}"] = s
    
df = df.T


In [ ]:
df.to_csv(f"{outfolder}/FWHM_data.csv")
df

In [ ]:
from matplotlib.ticker import NullFormatter
from matplotlib import pyplot, transforms
import matplotlib
from matplotlib.ticker import AutoMinorLocator
%matplotlib inline
import os
import copy
os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2018/bin/x86_64-darwin'
plt.rc('text', usetex=True)
plt.rc('font', weight='bold')
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.rm'] = 'Arial'
matplotlib.rcParams['mathtext.it'] = 'Arial:italic'
matplotlib.rcParams['mathtext.bf'] = 'Arial:bold'
matplotlib.rcParams['mathtext.tt'] = 'Arial'
matplotlib.rcParams['mathtext.cal'] = 'Arial'
matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

# plot for each time
fig, ax1 = plt.subplots(figsize=(8, 6))
fs = 20

ls_y = '-'
ls_z = '-'
m_y = 'o'
m_z = 's'

df['x'] = df.index
x = df['x'].astype(float).values # extraction offset
c = 'darkblue' # color
y = df['FWHM_z'].values

lbl = 'z'
ax1.plot(x, y, linestyle=ls_y, marker=m_y, color=c, label=lbl, markersize=10) # y

y = df['FWHM_y'].values
lbl = 'y'
c = 'darkorange' # color

ax1.plot(x, y, linestyle=ls_z, marker=m_z, color=c, label=lbl, markersize=10) # z

ax1.set_xlabel(r"\textbf{Offset extraction aperture [mm]}", fontsize=fs)
ax1.set_ylabel(r"\textbf{FWHM [mm]}", fontsize=fs)

ax1.tick_params(axis='y', labelsize=fs-2)
ax1.tick_params(axis='x', labelsize=fs-2)
# ax1.set_ylim(6, 9.5)
# ax1.set_xlim(10, 140)

# minor ticks x
minor_locator = AutoMinorLocator(2)
ax1.xaxis.set_minor_locator(minor_locator)
# minor ticks y
minor_locator = AutoMinorLocator(2)
ax1.yaxis.set_minor_locator(minor_locator)

my_handles, my_labels = ax1.get_legend_handles_labels()


# my_handles = [my_handles[0], my_handles[1], my_handles[2]]
# my_labels = ['90', '100', '120']
# l1 = plt.legend(my_handles, my_labels, loc=(0.56, 0.75), markerscale=0, fontsize=14)
# l1.set_title(r"\textbf{Distance aperture to target [cm]}", prop = {'size': 13})
# leg1 = plt.legend(loc='best', fontsize=fs-6)
# # change color of legend


from matplotlib.lines import Line2D

legend_elements = [Line2D([0], [0], marker='o', color='black', label='y',
                          markerfacecolor='black', markersize=10),
                   Line2D([0], [0], marker='s', color='black', label='z',
                          markerfacecolor='black', markersize=10)
                   ]

# l2 = plt.legend(handles=legend_elements,loc=(0.31, 0.81), handlelength=0, fontsize=14)
l2 = plt.legend(loc='upper left', fontsize=14)
l2.set_title(r"\textbf{FWHM along axis}", prop = {'size': 13})

# plt.gca().add_artist(l1)



ax1.grid(b=True, which='major', linestyle='-')
ax1.grid(b=True, which='minor', linestyle='--')


# ax1.set_xticks(np.arange(0,7,1))
plt.ylim(1, 6)
plt.xlim(7.75,15.255)

plt.tight_layout()
plt.savefig(f"{outfolder}{main_figname}.pdf")
plt.show()